<a href="https://colab.research.google.com/github/datarobot-community/mlops-examples/blob/master/Custom%20Model%20Examples/Boston%20PHousing/Main_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DRUM - Automated Model Serving Made Easy

 We'll get our hands dirty by 

* Building a simple regression model using Scikit
* Using DRUM for Batch Scoring
* Using DRUM to get a REST API endpoint
* Show a simple example app connected to the REST API
* H2O, Keras, XGBoost, and DataRobot
* Add a DataRobot remote agent if you are interested in further model monitoring


## Build a Model

In [16]:
!git clone https://github.com/datarobot-community/mlops-examples.git

fatal: destination path 'odsc-ml-drum' already exists and is not an empty directory.


In [17]:
!pip install -r /content/mlops-examples/'Custom Model Examples'/'Boston Housing'/colab-requirements.txt -q

In [18]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import pickle
import datetime

## load data

df = pd.read_csv('/content/mlops-examples/Custom Model Examples/Boston Housing/data/boston_housing.csv')
df.head()

## set features and target

X = df.drop('MEDV', axis=1)
y = df['MEDV']

## train the model
rf = RandomForestRegressor(n_estimators = 20)
rf.fit(X,y)

## serialize the model

with open('/content/mlops-examples/Custom Model Examples/Boston Housing/src/custom_model/rf.pkl', 'wb') as pkl:
    pickle.dump(rf, pkl)

# Batch Scoring with DRUM
<a id="setup_complete"></a>

At this point our model has been written to disk and we want to start making predictions with it.  To do this, we'll leverage DRUM and it's ability to natively handle our scikit learn model, all we need to do is tell DRUM where it resides as well as the data we wish to score.  

There are a lot of frameworks which DRUM supports nateively, but for those which DRUM doesn't support of these shelf, we'll just need to create some custom hooks so DRUM.  In this example, we'll highlight some very simple custom hooks, and will provide links to more complex examples.  

In [5]:
!drum score --code-dir /content/mlops-examples/'Custom Model Examples'/'Boston Housing'/src/custom_model --input /content/mlops-examples/'Custom Model Examples'/'Boston Housing'/data/boston_housing_inference.csv --output /content/mlops-examples/'Custom Model Examples'/'Boston Housing'/data/predictions.csv --target-type regression

/usr/local/lib/python3.6/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)
2020-11-16 17:56:56.524571: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [19]:
pd.read_csv("/content/mlops-examples/Custom Model Examples/Boston Housing/data/predictions.csv").head()

,Predictions
0,25.195
1,21.870
2,34.400
3,33.750
4,35.050


# Start the inference server locally

Batch scoring can be very useful, but the utility DRUM offers does not stop there.  We can also leverage DRUM to serve our model as a RESTful API endpoint.  The only thing that changes is the way we will structure the command - using the `server` mode instead of `score` model.  We'll also need to provide an address which is NOT in use.  

When starting the server, we'll use `subprocess.Popen` so we may interact with the server in this notebook

In [21]:
import subprocess
import requests
import pandas as pd
from io import BytesIO
import yaml
import time
import os
import datarobot as dr
from pprint import pprint

In [22]:
run_inference_server = ["drum",
              "server",
              "--code-dir","/content/mlops-examples/Custom Model Examples/Boston Housing/src/custom_model", 
              "--address", "0.0.0.0:6789", 
              "--show-perf",
              "--target-type", "regression",
              "--logging-level", "info",
              "--show-stacktrace",
              "--verbose"
              ]

In [23]:
inference_server = subprocess.Popen(run_inference_server, stdout=subprocess.PIPE)

## Ping the Server to make sure it is running

In [24]:
## confirm the server is running
time.sleep(5) ## snoozing before pinging the server to give it time to actually start
print('check status')
requests.request("GET", "http://0.0.0.0:6789/").content

check status


b'{"message":"OK"}\n'

## Send data to server for inference

The request must provide our dataset as form data.  In order to do so, we'll create a simple python function to pass the data over appropriately.  We'll leverage the same function in our simple flask app a little later.  

In [25]:
def score(data, port = "6789"):
    b_buf = BytesIO()
    b_buf.write(data.to_csv(index=False).encode("utf-8"))
    b_buf.seek(0)
  
    url = "http://localhost:{}/predict/".format(port)
    files = [
        ('X', b_buf)
    ]
    response = requests.request("POST", url, files = files, timeout=None, verify=False)
    return response

In [26]:
# %%timeit
scoring_data = pd.read_csv("/content/mlops-examples/Custom Model Examples/Boston Housing/data/boston_housing_inference.csv")
predictions = score(scoring_data).json() ## score entire dataset but only show first 5 records
pprint(predictions)

{'predictions': [26.345,
                 22.18,
                 34.64,
                 33.845,
                 35.32,
                 26.415,
                 22.52,
                 22.535,
                 16.685]}


In [28]:
requests.request("GET", "http://0.0.0.0:6789/").content

b'{"message":"OK"}\n'

In [29]:
requests.request("POST", "http://0.0.0.0:6789/shutdown/").content

b'Server shutting down...'

## Value Prop

One may ask, what is the benefit to be had here?  Well, first of, there is not need for me to write an api to get the model up and running.  Second, DRUM allows me to abstract the framework away (provided I'm using one that is natively supported, or I can write enough python so that DRUM understands how to hook up to the model.  

For example, I could hot swap models as I see fit (see exampels in `./src/other_models`)

While we will run through several other frameworks with in `score` you can bet they are supported in `server` mode as well!

#### H2O Mojo

In [30]:
!drum score --code-dir content/mlops-examples/'Custom Model Examples'/'Boston Housing'/src/other_models/h2o_mojo/regression --input content/mlops-examples/'Custom Model Examples'/'Boston Housing'/data/boston_housing_inference.csv --target-type regression


SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.
   Predictions
0    24.504000
1    22.492000
2    34.554001
3    34.420001
4    35.289001
5    28.394001
6    21.936000
7    23.451000
8    17.065000


#### Keras

In [31]:
!drum score --code-dir content/mlops-examples/'Custom Model Examples'/'Boston Housing'/src/other_models/python3_keras_joblib --input content/mlops-examples/'Custom Model Examples'/'Boston Housing'/data/boston_housing_inference.csv --target-type regression


2020-11-16 18:56:00.850533: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator SimpleImputer from version 0.23.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 0.23.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator Pipeline from version 0.23.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying t

#### XGBoost

Requires XGBoost

In [32]:
!drum score --code-dir /content/mlops-examples/'Custom Model Examples'/'Boston Housing'/src/other_models/python3_xgboost --input content/mlops-examples/'Custom Model Examples'/'Boston Housing'/data/boston_housing_inference.csv --target-type regression


/usr/local/lib/python3.6/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)
2020-11-16 18:56:07.501634: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
   Predictions
0    24.541843
1    21.260277
2    34.018497
3    32.569200
4    34.248066
5    27.282364
6    20.803959
7    19.645220
8    16.968880


#### DataRobot Codegen

In [33]:
!drum score --code-dir content/mlops-examples/'Custom Model Examples'/'Boston Housing'/src/other_models/dr_codegen --input content/mlops-examples/'Custom Model Examples'/'Boston Housing'/data/boston_housing_inference.csv --target-type regression


   Predictions
0    24.258228
1    24.258228
2    32.451515
3    32.451515
4    32.451515
5    24.258228
6    21.078378
7    13.107812
8    13.107812
